In [1]:
from pathlib import Path

import os
import sys

current_path = Path(os.getcwd())
sys.path.append(str(current_path))

import vectorian
vectorian.compile()

import spacy
nlp = spacy.load("en_core_web_sm")

import vectorian
from vectorian.embeddings import FastText

fasttext = FastText("en")

Matching checksum for /Users/arbeit/Projects/vectorian-2021/vectorian/core/cpp/core.cpp --> not compiling


In [2]:
import vectorian.utils as utils

token_mappings = {
    "tokenizer": [],
    "tagger": []
}

token_mappings["tokenizer"].append(utils.lowercase())
token_mappings["tokenizer"].append(utils.erase("W"))
token_mappings["tokenizer"].append(utils.alpha())

def filter_punct(t):
    return None if t["pos"] == "PUNCT" else t

token_mappings["tagger"].append(filter_punct)
token_mappings["tagger"].append(utils.rewrite({
    'pos': {
        'PROPN': 'NOUN'
    },
    'tag': {
        'NNP': 'NN',
        'NNPS': 'NNS',
    }
}))

In [21]:
from vectorian.importers import StringImporter
from vectorian.session import LabSession
from vectorian.metrics import CosineMetric, TokenSimilarityMetric
from vectorian.metrics import AlignmentSentenceMetric
from vectorian.alignment import WatermanSmithBeyer
from vectorian.metrics import CosineMetric, TokenSimilarityMetric, AlignmentSentenceMetric
from vectorian.alignment import WordMoversDistance

im = StringImporter(nlp)
doc = im("afternoon and morning sun")

session = LabSession(
    [doc],
    static_embeddings=[fasttext],
    token_mappings=token_mappings)

index = session.partition("sentence").index(AlignmentSentenceMetric(
    token_metric=TokenSimilarityMetric(fasttext, CosineMetric()),
    alignment=WordMoversDistance.wmd('vectorian')), nlp=nlp)

if False:
    index = session.partition("sentence").index(AlignmentSentenceMetric(
        token_metric=TokenSimilarityMetric(fasttext, CosineMetric()),
        alignment=WordMoversDistance.rwmd('vectorian')), nlp=nlp)

Importing <string>: 100%|██████████| 1/1 [00:00<00:00, 60.14it/s]


In [22]:
import json

def debug(hook, data):
    if hook == 'alignment_wmd_full_internal':
        print(data)

In [24]:
r = index.find("evening, oh evening star", n=1, debug=debug)

FloatProgress(value=0.0, layout=Layout(width='100%'), max=1.0)

{'s': {'id': [0, 1, 2, 3], 'text': ['afternoon', 'and', 'morning', 'sun']}, 't': {'id': [4, 5, 4, 6], 'text': ['evening', 'oh', 'evening', 'star']}, 'pos_to_vocab_s': array([0, 1, 2, 3], dtype=int16), 'pos_to_vocab_t': array([4, 5, 4, 6], dtype=int16), 'vocab_to_pos_s': {0: [0], 1: [1], 2: [2], 3: [3]}, 'vocab_to_pos_t': {4: [0, 2], 5: [1], 6: [3]}, 'bow_s': array([1., 1., 1., 1., 0., 0., 0.], dtype=float32), 'bow_t': array([0., 0., 0., 0., 2., 1., 1.], dtype=float32), 'D': array([[1.        , 1.        , 1.        , 1.        , 0.16250151,
        0.6226857 , 0.7048912 ],
       [1.        , 1.        , 1.        , 1.        , 0.59407264,
        0.31267238, 0.64379895],
       [1.        , 1.        , 1.        , 1.        , 0.2443018 ,
        0.60731065, 0.61894834],
       [1.        , 1.        , 1.        , 1.        , 0.508418  ,
        0.63617456, 0.50640804],
       [0.16250151, 0.59407264, 0.2443018 , 0.508418  , 1.        ,
        1.        , 1.        ],
       [0.622685

In [28]:
r

In [27]:
r.annotate('flow')